## Aim:

This algorithm aims for the computation of the evolution of stay-and-help strategy in a population composed of breeders, subordinates and waiters. 

## Libraries

In [3]:
using LinearAlgebra
using Plots
using LaTeXStrings
using Profile
using Random
using CSV
using DataFrames

## Parameters

In [11]:
#parameters

#survival
s_u = 0.5
s_x = 0.5
s_y = 0.5
s_jx = 0.5
s_jy = 0.5

#fecundity
b_x = 3
b_y = 4
phi = 0.1
M = 1
theta_x(x,y) = (1-phi)*x/(x+y)
theta_y(x,y) = (1-phi)*y/(x+y)
psi_x(x,y) = 1+phi+theta_x(x,y)
psi_y(x,y) = 1+phi+theta_y(x,y)

#competition
p_u = 0.5
p_j = 0.4
a = 2
p_u_tilde(x,y) = p_u/(1+a*(x+y))
p_j_tilde(x,y) = p_j/(1+a*(x+y))

#Cooperation
h_x = 0
h_y = 0
h_x_bar = 0
h_y_bar = 0
T_x(h_x) = 1-exp(-h_x*b_x)
T_y(h_y) = exp(-h_y*b_y)
reason = "reason"
parameters = Dict("b_x"=>b_x,"b_y"=>b_y,"s_u"=>s_u,"s_jx"=>s_jx,"s_jy"=>s_jy,"s_x"=>s_x,"s_y"=>s_y,"phi"=>phi,"M"=>M,"a"=>a,"p_u"=>p_u,"p_j"=>p_j,"h_x" =>h_x, "h_y" =>h_y,"h_x_bar" =>h_x_bar, "h_y_bar" =>h_y_bar,"reason" => reason)


Dict{String, Any} with 17 entries:
  "s_u"     => 0.5
  "h_x_bar" => 0
  "h_x"     => 0
  "p_u"     => 0.5
  "s_y"     => 0.5
  "a"       => 2
  "h_y_bar" => 0
  "M"       => 1
  "s_jx"    => 0.5
  "phi"     => 0.1
  "reason"  => "reason"
  "b_x"     => 3
  "h_y"     => 0
  "b_y"     => 4
  "s_jy"    => 0.5
  "s_x"     => 0.5
  "p_j"     => 0.4

## Simulation

In [13]:
# Wild type matrix
function A(x,y,parameters) 
    
    b_x = parameters["b_x"]
    b_y = parameters["b_y"]
    s_u = parameters["s_u"]
    s_jx = parameters["s_jx"]
    s_jy = parameters["s_jy"]
    s_x = parameters["s_x"]
    s_y = parameters["s_y"]
    p_u = parameters["p_u"]
    p_j = parameters["p_j"]
    a = parameters["a"]
    M = parameters["M"]
    phi = parameters["phi"]
    h_x = parameters["h_x"]
    h_y = parameters["h_y"]
    h_x_bar = parameters["h_x_bar"]
    h_y_bar = parameters["h_y_bar"]
    p_u_tilde(x,y) = p_u/(1+a*(x+y))
    p_j_tilde(x,y) = p_j/(1+a*(x+y))
    
    h_x = parameters["h_x"]
    h_y = parameters["h_y"]
    h_x_bar = parameters["h_x_bar"]
    h_y_bar = parameters["h_y_bar"]
    T_x(h_x) = 1-exp(-h_x*b_x)
    T_y(h_y) = exp(-h_y*b_y)
    
    return [s_u*(1-p_u_tilde(x,y)) 0 0 (1-h_x)*s_u*(1-p_u_tilde(x,y))+h_x*(T_x(h_x_bar)*s_jy+(1-T_x(h_x_bar))*s_jx)*(1-p_j_tilde(x,y)) (1-h_y)*s_u*(1-p_u_tilde(x,y))+h_y*(T_y(h_y_bar)*s_jx+(1-T_y(h_y_bar))*s_jy)*(1-p_j_tilde(x,y))
    ; s_u*p_u_tilde(x,y) (1-T_x(h_x_bar))*s_x T_y(h_y_bar)*s_x (1-h_x)*s_u*p_u_tilde(x,y)+h_x*(T_x(h_x_bar)*s_jy+(1-T_x(h_x_bar))*s_jx)*p_j_tilde(x,y) (1-h_y)*s_u*p_u_tilde(x,y)+h_y*(T_y(h_y_bar)*s_jx+(1-T_y(h_y_bar))*s_jy)*p_j_tilde(x,y)
    ; 0 T_x(h_x_bar)*s_y (1-T_y(h_y_bar))*s_y 0 0
    ; s_u*p_u_tilde(x,y)*b_x (1-T_x(h_x_bar))*s_x*b_x T_y(h_y_bar)*s_x*b_x ((1-h_x)*s_u*p_u_tilde(x,y)+h_x*(T_x(h_x_bar)*s_jy+(1-T_x(h_x_bar))*s_jx)*p_j_tilde(x,y))*b_x ((1-h_y)*s_u*p_u_tilde(x,y)+h_y*(T_y(h_y_bar)*s_jx+(1-T_y(h_y_bar))*s_jy)*p_j_tilde(x,y))*b_x
    ; 0 T_x(h_x_bar)*s_y*b_y (1-T_y(h_y_bar))*s_y*b_y 0 0
    ];
end

# Mutant matrix
function Mutant(x,y,parameters) 
    
    b_x = parameters["b_x"]
    b_y = parameters["b_y"]
    s_u = parameters["s_u"]
    s_jx = parameters["s_jx"]
    s_jy = parameters["s_jy"]
    s_x = parameters["s_x"]
    s_y = parameters["s_y"]
    p_u = parameters["p_u"]
    p_j = parameters["p_j"]
    a = parameters["a"]
    phi = parameters["phi"]
    p_u_tilde(x,y) = p_u/(1+a*(x+y))
    p_j_tilde(x,y) = p_j/(1+a*(x+y))
    theta_x(x,y) = (1-phi)*x/(x+y)
    theta_y(x,y) = (1-phi)*y/(x+y)
    
    psi_x(x,y) = theta_x(x,y)+1+phi
    psi_y(x,y) = theta_y(x,y)+1+phi
    T_x(h_x) = 1-exp(-h_x*b_x)
    T_y(h_y) = exp(-h_y*b_y)
    
    h_x = parameters["h_x"]
    h_y = parameters["h_y"]
    h_x_bar = parameters["h_x_bar"]
    h_y_bar = parameters["h_y_bar"]
    return [
        s_u*(1-p_u_tilde(x,y)) 0 0 (1-h_x)*s_u*(1-p_u_tilde(x,y))+h_x*(T_x(h_x_bar)*s_jy+(1-T_x(h_x_bar))*s_jx)*(1-p_j_tilde(x,y)) (1-h_y)*s_u*(1-p_u_tilde(x,y))+h_y*(T_y(h_y_bar)*s_jx+(1-T_y(h_y_bar))*s_jy)*(1-p_j_tilde(x,y))
    ; s_u*p_u_tilde(x,y) (1-T_x(h_x_bar))*s_x T_y(h_y_bar)*s_x (1-h_x)*s_u*p_u_tilde(x,y)+h_x*(T_x(h_x_bar)*s_jy+(1-T_x(h_x_bar))*s_jx)*p_j_tilde(x,y) (1-h_y)*s_u*p_u_tilde(x,y)+h_y*(T_y(h_y_bar)*s_jx+(1-T_y(h_y_bar))*s_jy)*p_j_tilde(x,y)
    ; 0 T_x(h_x_bar)*s_y (1-T_y(h_y_bar))*s_y 0 0
    ; s_u*p_u_tilde(x,y)*b_x/2*psi_x(x,y) (1-T_x(h_x_bar))*s_x*b_x/2*psi_x(x,y)+T_x(h_x_bar)*s_y*b_x/2*theta_x(x,y) T_y(h_y_bar)*s_x*b_x/2*psi_x(x,y)+(1-T_y(h_y_bar))*s_y*b_x/2*theta_x(x,y) (1-h_x)*s_u*p_u_tilde(x,y)*b_x/2*psi_x(x,y)+h_x*(T_x(h_x_bar)*s_jy+(1-T_x(h_x_bar))*s_jx)*p_j_tilde(x,y)*b_x/2*psi_x(x,y) (1-h_y)*s_u*p_u_tilde(x,y)*b_x/2*psi_x(x,y)+h_y*(T_y(h_y_bar)*s_jx+(1-T_y(h_y_bar))*s_jy)*p_j_tilde(x,y)*b_x/2*psi_x(x,y)
    ; s_u*p_u_tilde(x,y)*b_y/2*theta_y(x,y) (1-T_x(h_x_bar))*s_x*b_y/2*theta_y(x,y)+T_x(h_x_bar)*s_y*b_y/2*psi_y(x,y) T_y(h_y_bar)*s_x*b_y/2*theta_y(x,y)+(1-T_y(h_y_bar))*s_y*b_y/2*psi_y(x,y) (1-h_x)*s_u*p_u_tilde(x,y)*b_y/2*theta_y(x,y)+h_x*(T_x(h_x_bar)*s_jy+(1-T_x(h_x_bar))*s_jx)*p_j_tilde(x,y)*b_y/2*theta_y(x,y) (1-h_y)*s_u*p_u_tilde(x,y)*b_y/2*theta_y(x,y)+h_y*(T_y(h_y_bar)*s_jx+(1-T_y(h_y_bar))*s_jy)*p_j_tilde(x,y)*b_y/2*theta_y(x,y)
        ];
end;

function equilibrium_mu(parameters)
    mu = rand!(zeros(5,1))
    while sum(abs.(mu - A(mu[2], mu[3],parameters)*mu)) > 1e-7
        mu = A(mu[2], mu[3],parameters)*mu
    end
    return mu
end;

function equilibrium_nu(parameters,mu)
    nu = rand!(zeros(1,5))
    while sum(abs.(nu - nu*A(mu[2], mu[3],parameters))) >1e-7
        nu = nu*A(mu[2], mu[3],parameters)
        nu /= sum(nu)
    end
    return nu
end;

function equilibrium_nu_M(parameters,mu)
    nu = rand!(zeros(1,5))
    while sum(abs.(nu - nu*Mutant(mu[2], mu[3],parameters))) >1e-7
        nu = nu*Mutant(mu[2], mu[3],parameters)
        nu /= sum(nu)
    end
    return nu
end;

function R_jp(phi)
    
    R_jp = 1/2*(1+phi)

    return  R_jp
end

function R_siblings_x(parameters)
    b_x = parameters["b_x"] 
    M = parameters["M"]
    phi = parameters["phi"]
    R_siblings_x = (1-1/b_x)*(phi^2+phi*(1-phi)/2+(1-phi)^2*(1/4+1/(4*M)))+1/b_x
    return  R_siblings_x
end
function R_siblings_y(parameters)
    M = parameters["M"]
    phi = parameters["phi"]
    b_y = parameters["b_y"]
    R_siblings_y = (1-1/b_y)*(phi^2+phi*(1-phi)/2+(1-phi)^2*(1/4+1/(4*M)))+1/b_y

    return  R_siblings_y
end




function R_0(parameters)
    b_x = parameters["b_x"]
    b_y = parameters["b_y"]
    s_u = parameters["s_u"]
    s_jx = parameters["s_jx"]
    s_jy = parameters["s_jy"]
    s_x = parameters["s_x"]
    s_y = parameters["s_y"]
    p_u = parameters["p_u"]
    p_j = parameters["p_j"]
    a = parameters["a"]
    p_u_tilde(x,y) = p_u/(1+a*(x+y))
    p_j_tilde(x,y) = p_j/(1+a*(x+y))
    M = parameters["M"]
    phi = parameters["phi"]
    h_x = parameters["h_x"]
    h_y = parameters["h_y"]
    h_x_bar = parameters["h_x_bar"]
    h_y_bar = parameters["h_y_bar"]
    nu_u = 1
    nu_jx = (1 - h_x) + h_x*((1 − T_x(h_x_bar))*s_jx + T_x(h_x_bar)*s_jy )*(1 − p_j+p_j*(1-s_u*(1-p_u))/s_u/p_u)
    nu_jy = (1 - h_y) + h_y*((1 − T_y(h_y_bar))*s_jy + T_y(h_y_bar)*s_jx )*(1 − p_j+p_j*(1-s_u*(1-p_u))/s_u/p_u)
    nu_y = 1/(1-s_y*(1-T_y(h_y_bar)))*(((1-T_y(h_y_bar))*s_y*b_y*nu_jy)+T_y(h_y_bar)*s_x*(1-s_u*(1-p_u))/s_u/p_u)
    nu_x = (1-T_x(h_x_bar))*s_x*(1-s_u*(1-p_u))/s_u/p_u+T_x(h_x_bar)*s_y*(nu_y+b_y*nu_jy)
    R_0 = s_u*(1-p_u)*nu_u+s_u*p_u*(nu_x+b_x*nu_jx)
    return R_0
end



R_0 (generic function with 1 method)

### Equilibrium computation

In [4]:
function equilibrium()
    pop = transpose([1 1 1 1 1])
    for t in t_0:t_max
        pop = A(pop[3], pop[4],parameters)*pop
    end
    return pop
end;
println(equilibrium())

[3.8888886790034882; 3.33333322501637; 0.0; 9.99999967504911; 0.0;;]


In [8]:
parameters["h_x"] = 0.3
parameters["h_y"] = 0.3
parameters["h_x_bar"] = 0.3
parameters["h_y_bar"] = 0.3
mu = equilibrium_mu(parameters)
println(equilibrium_nu(parameters,mu))
println(equilibrium_nu_M(parameters,mu))

[0.06976575770046821 0.3948858539015825 0.3979098456310402 0.06871927138345459 0.06871927138345459]
[0.07094973511474201 0.3937939333625197 0.39548535367114374 0.06988548892579723 0.06988548892579723]


## Evolution

In [14]:
function Fitness_comput(parameters)
    d_hx = 0.001
    d_hx_bar = 0.001
    d_hy = 0.001
    d_hy_bar = 0.001
    dW_x = 0
    dW_y = 0
    # Find eigenvectors: mu, nu

    mu = equilibrium_mu(parameters)
    nu = equilibrium_nu_M(parameters, mu)

    # h_x, h_x_bar
    parameters["h_x"] += d_hx
    W_indiv_hx_up = nu*Mutant(mu[2], mu[3],parameters)*[0,0,0,mu[4],0]
    parameters["h_x"] -= 2*d_hx
    W_indiv_hx_down = nu*Mutant(mu[2], mu[3],parameters)*[0,0,0,mu[4],0]

    parameters["h_x"] += d_hx
    parameters["h_x_bar"] +=d_hx_bar
    
    W_parent_hx_up = nu*Mutant(mu[2], mu[3],parameters)*[0,mu[2],0,0,0]
    W_siblings_hx_up = nu*Mutant(mu[2], mu[3],parameters)*[0,0,0,mu[4],0]

    parameters["h_x_bar"] -= 2*d_hx_bar
    W_parent_hx_down = nu*Mutant(mu[2], mu[3],parameters)*[0,mu[2],0,0,0]
    W_siblings_hx_down = nu*Mutant(mu[2], mu[3],parameters)*[0,0,0,mu[4],0]


    parameters["h_x_bar"] +=d_hx_bar

    #control by subordinate
    W_indiv_hx = W_indiv_hx_up-W_indiv_hx_down
    W_parent_hx = R_jp(parameters["phi"])*(W_parent_hx_up-W_parent_hx_down)
    W_siblings_hx = R_siblings_x(parameters)*(W_siblings_hx_up-W_siblings_hx_down)

    #control by parent
    #W_parent_hx = (W_parent_hx_up-W_parent_hx_down)
    #W_indiv_hx = R_jp(parameters["phi"])*(W_indiv_hx_up-W_indiv_hx_down)
    #W_siblings_hx = R_jp(parameters["phi"])*(W_siblings_hx_up-W_siblings_hx_down)
    dW_x = W_parent_hx[1]+W_indiv_hx[1]+W_siblings_hx[1]

    # h_y, h_y_bar

    parameters["h_y"] += d_hy
    W_indiv_hy_up = nu*Mutant(mu[2], mu[3],parameters)*[0,0,0,0,mu[5]]
    parameters["h_y"] -= 2*d_hy
    W_indiv_hy_down = nu*Mutant(mu[2], mu[3],parameters)*[0,0,0,0,mu[5]]

    parameters["h_y"] += d_hy
    parameters["h_y_bar"] += d_hy_bar
    
    W_parent_hy_up = nu*Mutant(mu[2], mu[3],parameters)*[0,0,mu[3],0,0]
    W_siblings_hy_up = nu*Mutant(mu[2], mu[3],parameters)*[0,0,0,0,mu[5]]

    parameters["h_y_bar"] -= 2*d_hy_bar
    W_parent_hy_down = nu*Mutant(mu[2], mu[3],parameters)*[0,0,mu[3],0,0]
    W_siblings_hy_down = nu*Mutant(mu[2], mu[3],parameters)*[0,0,0,0,mu[5]]
    parameters["h_y_bar"] += d_hy_bar
    
    
    #control by subordinate
    W_indiv_hy = W_indiv_hy_up-W_indiv_hy_down
    W_parent_hy = R_jp(parameters["phi"])*(W_parent_hy_up-W_parent_hy_down)
    W_siblings_hy = R_siblings_y(parameters)*(W_siblings_hy_up-W_siblings_hy_down)
    #control by parent
    #W_indiv_hy = R_jp(parameters["phi"])*(W_indiv_hy_up-W_indiv_hy_down)
    #W_parent_hy = (W_parent_hy_up-W_parent_hy_down)
    #W_siblings_hy = R_jp(parameters["phi"])*(W_siblings_hy_up-W_siblings_hy_down)
    dW_y = W_parent_hy[1]+W_indiv_hy[1]+W_siblings_hy[1]
    
   
    return dW_x, dW_y
   
    
end;



In [15]:
function ESS(parameters)
    dW_x = 1
    dW_y = 1
    i=0
    h_x_mem = 1
    h_y_mem = 1
    reason = "no found"
    while abs(dW_x) > 1e-8 || abs(dW_y) > 1e-8
      
        i += 1
        #If taking too much time
        if i > 100000
            reason = "i>100000"
            break
            println("too long")
        end
        
        
        #Same values previously computed
        if parameters["h_x"]==h_x_mem && parameters["h_y"]==h_y_mem
            reason = "stagnation"
            break
        end
        
        
        dW_x,dW_y = Fitness_comput(parameters)
        
        #Loading previous values
        h_x_mem = parameters["h_x"]
        h_y_mem = parameters["h_y"]
    
        #New values
        parameters["h_x"] += dW_x
        parameters["h_x_bar"] += dW_x
        
        parameters["h_y"] += dW_y
        parameters["h_y_bar"] += dW_y
        
        #Boundaries conditions
        parameters["h_x"] = clamp(parameters["h_x"],0,1)
        parameters["h_y"] = clamp(parameters["h_y"],0,1)
        parameters["h_x_bar"] = parameters["h_x"]
        parameters["h_y_bar"] = parameters["h_y"]
         
    end
    if abs(dW_x) < 1e-8 || abs(dW_y) < 1e-10
        reason = "ess found"
    end
    parameters["h_x"] = clamp(parameters["h_x"],0,1)
    parameters["h_y"] = clamp(parameters["h_y"],0,1)
    parameters["h_x_bar"] = parameters["h_x"]
    parameters["h_y_bar"] = parameters["h_y"]
    parameters["reason"] = reason
    return parameters

end


ESS (generic function with 1 method)

## Simulation

In [ ]:
#Group augmentation on breeder fecundity
parameters["s_x"] = 0.4
parameters["s_y"] = 0.4
parameters["s_u"] = 0.4
parameters["s_jx"] = 0.4
parameters["s_jy"] = 0.4
parameters["p_j"] = 0.5
parameters["p_u"] = 0.6
parameters["b_x"] = 3
parameters["b_y"] = 3
parameters["phi"] = 0.1

N = 5
h_x_csv = zeros(N^2)
h_y_csv = zeros(N^2)
P_csv = zeros(N^2)
P2_csv = zeros(N^2)
b_csv = zeros(N^2)
db_csv = zeros(N^2)
GA_csv = zeros(N^2)
reason_csv = Array{String}(undef, N^2)
i = 1

for b_x in range(2,7,N)
    parameters["b_x"] = b_x
   
    for db = range(0,2,N)
        parameters["b_y"] = b_x+db

        parameters["h_x"] = 0
        parameters["h_y"] = 0
        parameters["h_x_bar"] = 0
        parameters["h_y_bar"] = 0
        if R_0(parameters) > 1.02

            eq = equilibrium_mu(parameters)
            P = parameters["s_u"]*parameters["p_u"]/(1+a*(eq[2]+eq[3]))/(1-parameters["s_u"]*(1-parameters["p_u"]/(1+a*(eq[2]+eq[3]))))           
            
            parameters = ESS(parameters)

            eq = equilibrium_mu(parameters)
            mu = eq
            nu = equilibrium_nu_M(parameters,mu)
            P2 = parameters["s_u"]*parameters["p_u"]/(1+a*(eq[2]+eq[3]))/(1-parameters["s_u"]*(1-parameters["p_u"]/(1+a*(eq[2]+eq[3]))))
            
            h_x_csv[i] = parameters["h_x"]
            h_y_csv[i] = parameters["h_y"]
            GA_csv[i] = (1-p_j_tilde(mu[2],mu[3]))*nu[1]+p_j_tilde(mu[2],mu[3])*(nu[2]+(1+phi)*b_x/2*nu[4]+(1-phi)*mu[2]/(mu[2]+mu[3])*b_x/2*nu[4]+(1-phi)*mu[3]/(mu[2]+mu[3])*b_y/2*nu[5])
            P_csv[i] = P
            P2_csv[i] = P2
            b_csv[i] = b_x
            db_csv[i] = db
            
        
        end
        reason_csv[i] = parameters["reason"]
        h_x_csv[i] = parameters["h_x"]
        h_y_csv[i] = parameters["h_y"]
        b_csv[i] = b_x
        db_csv[i] = db
        i+=1
    end
end
csv_data = DataFrame(db = db_csv, GA=GA_csv, P = P_csv, P2 = P2_csv, b = b_csv, h_x = h_x_csv, h_y = h_y_csv ,parameters = parameters)
CSV.write("result_p_b.csv",csv_data)


In [39]:
ESS(parameters)

LoadError: MethodError: [0mCannot `convert` an object of type [92mString[39m[0m to an object of type [91mReal[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::Gray24[39m) where T<:Real at C:\Users\Alan\.julia\packages\ColorTypes\6m8P7\src\conversions.jl:114
[0m  convert(::Type{T}, [91m::Gray[39m) where T<:Real at C:\Users\Alan\.julia\packages\ColorTypes\6m8P7\src\conversions.jl:113
[0m  convert(::Type{T}, [91m::Base.TwicePrecision[39m) where T<:Number at C:\Users\Alan\AppData\Local\Programs\Julia-1.7.1\share\julia\base\twiceprecision.jl:262
[0m  ...

In [29]:
parameters["b_x"] = rand(1)[1]*5+3
parameters["b_y"] = parameters["b_x"]+rand(1)[1]
parameters = ESS(parameters)
println(parameters["b_x"]*parameters["h_x"])
println(parameters["b_y"]*parameters["h_y"])


0.0
1.2593754630534482


In [35]:
#parameters

#survival
s_u = 0.5
s_x = 0.5
s_y = 0.5
s_jx = 0.5
s_jy = 0.5

#fecundity
b_x = 3
b_y = 4
phi = 0.1
M = 1
theta_x(x,y) = (1-phi)*x/(x+y)
theta_y(x,y) = (1-phi)*y/(x+y)
psi_x(x,y) = 1+phi+theta_x(x,y)
psi_y(x,y) = 1+phi+theta_y(x,y)

#competition
p_u = 0.5
p_j = 0.5
a = 2
p_u_t(x,y) = p_u/(1+a*(x+y))
p_j_t(x,y) = p_j/(1+a*(x+y))

#Cooperation
h_x = 0.5
h_y = 0.5
T_x(h_x) = 1-exp(-h_x*b_x)
T_y(h_y) = exp(-h_y*b_y)

parameters = Dict("b_x"=>b_x,"b_y"=>b_y,"s_u"=>s_u,"s_jx"=>s_jx,"s_jy"=>s_jy,"s_x"=>s_x,"s_y"=>s_y,"phi"=>phi,"M"=>M,"a"=>a,"p_u"=>p_u,"p_j"=>p_j,"h_x" =>h_x, "h_y" =>h_y,"h_x_bar" =>h_x, "h_y_bar" =>h_y, )

mu = equilibrium_mu(parameters)
equilibrium_nu_M(parameters,mu)

1×5 Matrix{Float64}:
 0.0972319  0.352924  0.35538  0.0972319  0.0972319

In [42]:
a=-10
sign(a)

-1